In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import gradio as gr
from transformers import pipeline, AutoTokenizer, TextStreamer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential = True)

In [ ]:
# CONSTANTS
LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

system_prompt = """
  You are an AI assistant who generates syntethic test data set for various different kinds of fileds. You should create synthetic yet believable datasets that can be valueable for
  different kinds of businesses or research purpose. Just answer to the datasets request with near-real-world data in json format.
  For example, let's say user asks for fake university students dataset for study habits, you might reply with:
  {
    "dataset": [
      {
        "student-id": "1",
        "courses":...,
        "hours_studied":...,
        "hours_in_sports":...,
        "grades":...
      },
      .
      .
      .
    ]
  }
Improvise to make things better if you see fit.
"""

In [ ]:
# Creating the model

def generate(model, data_generation_topic):
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": data_generation_topic}
  ]
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map='auto',
                                               quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
  del tokenizer, streamer, model, inputs, outputs
  torch.cuda.empty_cache()

In [ ]:
generate(LLAMA, "Generate data on spending habits of 5 random people from Nepal")